In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import glob
import skimage.io as io
from skimage import data_dir
import split_folders
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from skimage.io import imread_collection
import random
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#this function reads in images and then flattens them into a dataframe
def flat_images(path):
    images=io.ImageCollection(path)
    flat_images=[]
    for i in range(len(images)):
        img=images[i]
        img1=img.flatten()
        flat_images.append(img1)
    df=pd.DataFrame(flat_images)
    return df

In [3]:
#flattens ditylum images
path= r'C:\Users\vrm8601\Documents\Pre Prac\ditsample\*.tif'
df_1=flat_images(path)
df_1=df_1.fillna(0)
df_1['label']=1

In [4]:
#flattens non-ditylum images
#path1= r'C:\Users\vrm8601\Documents\Pre Prac\0_notditylum\0\*.tif'
path1= r'C:\Users\vrm8601\Documents\Pre Prac\notsample\*.tif'
df_0=flat_images(path1)
df_0=df_0.fillna(0)
df_0['label']=0

In [5]:
#puts both sets of images in same dataframe
df=df_1.append(df_0)
df.reset_index(inplace=True,drop=True)
df=df.fillna(0)

In [6]:
# create test and train data (80/20 split)
X=df.drop(['label'],axis=1)
y=df.label
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [ ]:
# tip: always run PCA after the split only on train data
pca=PCA()
pca.fit(X_train)
csum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(csum>=.99)+1

pca=PCA(n_components=d)
X_train_reduced=pca.fit_transform(X_train)
#print(X_train.shape, X_train_reduced.shape) 

reduce test set too for metrics later - not in the model itself
X_test_reduced=pca.transform(X_test)
#print(X_test.shape, X_test_reduced.shape)

In [7]:
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train_transform = scaler.transform(X_train_reduced)
X_test_transform = scaler.transform(X_test_reduced)

In [10]:
error_rate = []

# Will take some time
for i in range(1,12):
    # create a knn classifier with n_neighbors equals i
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train_transform,y_train)
    pred_i = knn.predict(X_test_transform)
    # calculate the error rate and append to the error_rate list
    error_rate.append(np.mean(pred_i != y_test))

In [11]:
print(error_rate)

[0.25, 0.25, 0.25, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75]
